# Noisy Labels in Dataset (gcommand)

dataset의 labelling이 잘못 되었을 떄의  효과를 보기 위해.

In [6]:
import pandas as pd
import numpy as np
import random

In [4]:
gco_si = pd.read_csv("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/gcommand_si.csv")
gco_sv = pd.read_csv("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/gcommand_sv.csv")

In [25]:
# split gco_si into in_dist and out_dist
gco_si_spks = gco_si.spk.unique().tolist()
even_num = len(gco_si_spks)//2
in_spks = np.random.choice(gco_si_spks, size=even_num, replace=False) 
out_spks = np.random.choice(list(set(gco_si_spks) - set(in_spks)), size=even_num, replace=False)

In [65]:
gco_si_in = gco_si[gco_si.spk.isin(in_spks)]
gco_si_out = gco_si[gco_si.spk.isin(out_spks)]

gco_si_in.loc[:, 'label'] = gco_si_in.groupby("spk").ngroup()
gco_si_out.loc[:, 'label'] = gco_si_out.groupby("spk").ngroup()

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
gco_si_in.to_csv("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/noisy_train/gco_si_in.csv", index=False)

## out-dist injection

In [43]:
injected_out = gco_si_out.groupby("label", group_keys=False).apply(lambda x: x.sample(n=5))
# injected noise are used for train set.
injected_out.loc[:, 'set'] = 'train'

In [88]:
# out-dist injected
out_injected_si = pd.concat([gco_si_in, injected_out])
out_injected_si = out_injected_si.sort_values('label')

## in-dist random labels

In [82]:
# in-dist label random shuffle
seleted_samples = gco_si_in.groupby("label", group_keys=False).apply(lambda x: x.sample(frac=0.1))

In [83]:
label_dist = gco_si_in.label.value_counts().values
label_dist_norm = label_dist / label_dist.sum()

In [84]:
random_label = np.random.randint(0, even_num, size=len(seleted_samples))

In [85]:
random_label_si = gco_si_in.copy()
random_label_si.loc[seleted_samples.index, 'label'] = random_label

In [86]:
random_label_si.to_csv("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/noisy_train/gco_randomLabel_si.csv", index=False)